In [34]:
import numpy as np 
import pandas as pd 
import re
import json

import os
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

#!pip install emoji --upgrade
import emoji

#!pip install tweet-preprocessor
import preprocessor as p

from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

In [35]:
# Read the data
file_path = "/Users/olgapodolska/Desktop/UA_War_my/resources/aug_twitter_data.csv"
war_df = pd.read_csv(file_path)

war_df.head()

/Users/olgapodolska/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3457: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,10,11,12) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,following,followers,totaltweets,usercreatedts,tweetcreatedts,retweetcount,text,hashtags,language,favorite_count,is_retweet,is_quote_status,extractedts
0,51,3362,71331,2011-09-08 02:34:54.000000,2022-08-01 00:00:00.000000,0,The #RussiaUkraine conflict is being cast in b...,"[{'text': 'RussiaUkraine', 'indices': [4, 18]}...",en,0.0,False,False,2022-08-01 02:30:13.601013
1,2964,4669,119795,2017-07-06 15:28:29.000000,2022-08-01 00:00:00.000000,560,Remember when #NATO smashed #Yugoslavia into s...,"[{'text': 'NATO', 'indices': [14, 19]}, {'text...",en,0.0,True,False,2022-08-01 02:30:13.573250
2,49,8,112,2021-04-04 17:47:00.000000,2022-08-01 00:00:01.000000,0,"Hi #NATO , you have to understand that we are ...","[{'text': 'NATO', 'indices': [3, 8]}]",en,0.0,False,False,2022-08-01 06:59:10.496078
3,137,114789,315096,2010-08-20 11:19:22.000000,2022-08-01 00:00:01.000000,0,The world is in dire straits as it is not equi...,"[{'text': 'UkraineCrisis', 'indices': [133, 14...",en,1.0,False,False,2022-08-01 01:44:58.884242
4,27,3059,13665,2017-01-06 10:17:40.000000,2022-08-01 00:00:01.000000,0,Will the #sanctions imposed on #Russia cause a...,"[{'text': 'sanctions', 'indices': [9, 19]}, {'...",en,0.0,False,False,2022-08-01 05:13:47.957650


In [36]:
# get shape of the DataFrame
print(f"{war_df.shape[0]} rows and {war_df.shape[1]} columns")

1255638 rows and 13 columns


In [37]:
# Reduce the data: take English tweets only
war_df=war_df.loc[war_df['language']=='en']
war_df.head()

,following,followers,totaltweets,usercreatedts,tweetcreatedts,retweetcount,text,hashtags,language,favorite_count,is_retweet,is_quote_status,extractedts
0,51,3362,71331,2011-09-08 02:34:54.000000,2022-08-01 00:00:00.000000,0,The #RussiaUkraine conflict is being cast in b...,"[{'text': 'RussiaUkraine', 'indices': [4, 18]}...",en,0.0,False,False,2022-08-01 02:30:13.601013
1,2964,4669,119795,2017-07-06 15:28:29.000000,2022-08-01 00:00:00.000000,560,Remember when #NATO smashed #Yugoslavia into s...,"[{'text': 'NATO', 'indices': [14, 19]}, {'text...",en,0.0,True,False,2022-08-01 02:30:13.573250
2,49,8,112,2021-04-04 17:47:00.000000,2022-08-01 00:00:01.000000,0,"Hi #NATO , you have to understand that we are ...","[{'text': 'NATO', 'indices': [3, 8]}]",en,0.0,False,False,2022-08-01 06:59:10.496078
3,137,114789,315096,2010-08-20 11:19:22.000000,2022-08-01 00:00:01.000000,0,The world is in dire straits as it is not equi...,"[{'text': 'UkraineCrisis', 'indices': [133, 14...",en,1.0,False,False,2022-08-01 01:44:58.884242
4,27,3059,13665,2017-01-06 10:17:40.000000,2022-08-01 00:00:01.000000,0,Will the #sanctions imposed on #Russia cause a...,"[{'text': 'sanctions', 'indices': [9, 19]}, {'...",en,0.0,False,False,2022-08-01 05:13:47.957650


In [38]:
# get shape of the DataFrame
print(f"{war_df.shape[0]} rows and {war_df.shape[1]} columns")

1050058 rows and 13 columns


In [39]:
# Change the dtypes of usercreatedts, tweetcreatedts, and extractedts to datetime64 for easier operation later
war_df["usercreatedts"] = pd.to_datetime(war_df["usercreatedts"])
war_df["tweetcreatedts"] = pd.to_datetime(war_df["tweetcreatedts"])
war_df["extractedts"] = pd.to_datetime(war_df["extractedts"])

# check dtypes
war_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1050058 entries, 0 to 1255637
Data columns (total 13 columns):
 #   Column           Non-Null Count    Dtype         
---  ------           --------------    -----         
 0   following        1050058 non-null  object        
 1   followers        1050058 non-null  object        
 2   totaltweets      1050058 non-null  object        
 3   usercreatedts    1050058 non-null  datetime64[ns]
 4   tweetcreatedts   1050058 non-null  datetime64[ns]
 5   retweetcount     1050058 non-null  object        
 6   text             1050058 non-null  object        
 7   hashtags         1050058 non-null  object        
 8   language         1050058 non-null  object        
 9   favorite_count   1050058 non-null  float64       
 10  is_retweet       1050058 non-null  object        
 11  is_quote_status  1050058 non-null  object        
 12  extractedts      1050058 non-null  datetime64[ns]
dtypes: datetime64[ns](3), float64(1), object(9)
memory usage:

# Reducing the data

In [63]:
# Reduce the data: take the usercreatedts more then 5 years

time_df = war_df[(war_df['usercreatedts'] < "2010-01-01")]
print(f"{time_df.shape[0]} rows and {time_df.shape[1]} columns")

96683 rows and 13 columns
